In [7]:
%reload_ext tensorboard

In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.datasets.mnist import load_data
data = pd.read_csv("train.csv")

X_col = [col for col in data.columns if col not in ['label']]
(X_train_full, y_train_full), (X_test_full, y_test_full) = load_data()
X_train_full = np.reshape(X_train_full, (-1, 784))
X_test_full = np.reshape(X_test_full, (-1, 784))
X_full = data[X_col]
y_full = data['label']
X_valid, X_train = X_full/255.0, X_train_full/255.0
y_valid, y_train = y_full, y_train_full

X_test_full.shape

(10000, 784)

In [3]:
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

def build_model(n_hidden=5, n_neurons=256):
    model = keras.models.Sequential()
    model.add(Dense(n_neurons, activation='elu', kernel_initializer='he_normal', input_shape=[784]))
    keras.layers.Dropout(rate=0.4)
    for layer in range(n_hidden-1):
        keras.layers.BatchNormalization()
        model.add(Dense(n_neurons, activation='elu', kernel_initializer='he_normal'))
        keras.layers.Dropout(rate=0.5)
    keras.layers.BatchNormalization()
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy',
                 optimizer='Nadam',
                 metrics=['accuracy'])
    return model

model = build_model()

In [11]:
from datetime import datetime 

logdir = "logs/scalars" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [5]:
#early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, 
#                                                 restore_best_weights=True)

checkpoint_cb = keras.callbacks.ModelCheckpoint("keras_mnist.h5",
save_best_only=True)

history = model.fit(X_train, y_train, epochs=100,
                    validation_data=(X_valid, y_valid),
                    callbacks=[tensorboard_callback, checkpoint_cb])

Epoch 1/100
1875/1875 [==============================] - 54s 29ms/step - loss: 0.2376 - accuracy: 0.9276 - val_loss: 0.1550 - val_accuracy: 0.9541
Epoch 2/100
1875/1875 [==============================] - 49s 26ms/step - loss: 0.1306 - accuracy: 0.9608 - val_loss: 0.1176 - val_accuracy: 0.9647
Epoch 3/100
1875/1875 [==============================] - 48s 25ms/step - loss: 0.1084 - accuracy: 0.9682 - val_loss: 0.1004 - val_accuracy: 0.9693
Epoch 4/100
1875/1875 [==============================] - 47s 25ms/step - loss: 0.0910 - accuracy: 0.9729 - val_loss: 0.0547 - val_accuracy: 0.9826
Epoch 5/100
1875/1875 [==============================] - 55s 29ms/step - loss: 0.0821 - accuracy: 0.9751 - val_loss: 0.0811 - val_accuracy: 0.9758
Epoch 6/100
1875/1875 [==============================] - 42s 22ms/step - loss: 0.0708 - accuracy: 0.9791 - val_loss: 0.0601 - val_accuracy: 0.9812
Epoch 7/100
1875/1875 [==============================] - 58s 31ms/step - loss: 0.0620 - accuracy: 0.9818 - val_loss: 0

1875/1875 [==============================] - 33s 18ms/step - loss: 0.0187 - accuracy: 0.9969 - val_loss: 0.0468 - val_accuracy: 0.9932
Epoch 57/100
1875/1875 [==============================] - 34s 18ms/step - loss: 0.0161 - accuracy: 0.9971 - val_loss: 0.0381 - val_accuracy: 0.9952
Epoch 58/100
1875/1875 [==============================] - 38s 20ms/step - loss: 0.0192 - accuracy: 0.9968 - val_loss: 0.0578 - val_accuracy: 0.9965
Epoch 59/100
1875/1875 [==============================] - 32s 17ms/step - loss: 0.0932 - accuracy: 0.9966 - val_loss: 0.0436 - val_accuracy: 0.9935
Epoch 60/100
1875/1875 [==============================] - 28s 15ms/step - loss: 0.0094 - accuracy: 0.9979 - val_loss: 0.0561 - val_accuracy: 0.9953
Epoch 61/100
1875/1875 [==============================] - 37s 20ms/step - loss: 0.0171 - accuracy: 0.9975 - val_loss: 0.0434 - val_accuracy: 0.9966
Epoch 62/100
1875/1875 [==============================] - 38s 20ms/step - loss: 0.0148 - accuracy: 0.9974 - val_loss: 0.0461 

In [12]:
%tensorboard --logdir logs/scalars

Reusing TensorBoard on port 6006 (pid 81878), started 0:00:47 ago. (Use '!kill 81878' to kill it.)

In [13]:
model = keras.models.load_model("keras_mnist.h5")

model_new = keras.models.Sequential(model.layers[:-1])
model_new.add(keras.layers.Dense(10, activation='softmax'))

for layer in model_new.layers[:-1]:
    layer.trainable = False
    
model_new.compile(loss='sparse_categorical_crossentropy',
                 optimizer='Nadam',
                 metrics=['accuracy'])

X_Valid, X_Train = X_full[36000:]/255.0, X_full[:36000]/255.0
y_Valid, y_Train = y_full[36000:], y_full[:36000]

In [14]:
history = model_new.fit(X_Train, y_Train, epochs=10,
                    validation_data=(X_Valid, y_Valid))

Epoch 1/10
1125/1125 [==============================] - 7s 6ms/step - loss: 0.3247 - accuracy: 0.9597 - val_loss: 0.0441 - val_accuracy: 0.9933
Epoch 2/10
1125/1125 [==============================] - 6s 5ms/step - loss: 0.0246 - accuracy: 0.9943 - val_loss: 0.0457 - val_accuracy: 0.9937
Epoch 3/10
1125/1125 [==============================] - 8s 7ms/step - loss: 0.0203 - accuracy: 0.9948 - val_loss: 0.0430 - val_accuracy: 0.9942
Epoch 4/10
1125/1125 [==============================] - 7s 6ms/step - loss: 0.0187 - accuracy: 0.9952 - val_loss: 0.0431 - val_accuracy: 0.9943
Epoch 5/10
1125/1125 [==============================] - 5s 4ms/step - loss: 0.0175 - accuracy: 0.9956 - val_loss: 0.0418 - val_accuracy: 0.9950
Epoch 6/10
1125/1125 [==============================] - 5s 4ms/step - loss: 0.0175 - accuracy: 0.9954 - val_loss: 0.0438 - val_accuracy: 0.9942
Epoch 7/10
1125/1125 [==============================] - 3s 3ms/step - loss: 0.0169 - accuracy: 0.9958 - val_loss: 0.0441 - val_accuracy:

In [15]:
test = pd.read_csv("test.csv")

In [16]:
y_test = model_new.predict_classes(test/255)

output = pd.DataFrame({"ImageId": test.index+1, 'Label':y_test})

print(model_new.evaluate(X_train_full/255, y_train_full))
print(model.evaluate(X_train_full/255, y_train_full))
print(model_new.evaluate(X_full/255, y_full))
print(model.evaluate(X_full/255, y_full))

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0097 - accuracy: 0.9971
[0.009663091972470284, 0.9971166849136353]
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0115 - accuracy: 0.9967
[0.011515735648572445, 0.9967333078384399]
1313/1313 [==============================] - 7s 5ms/step - loss: 0.0195 - accuracy: 0.9958
[0.01945994608104229, 0.9958333373069763]
1313/1313 [==============================] - 4s 3ms/step - loss: 0.0257 - accuracy: 0.9948
[0.0256603192538023, 0.9948333501815796]


In [17]:
output.to_csv('sub.csv', index=False)